# Demo: Retrieving basic entities

## Import python modules

In [ ]:
import graphsense
from graphsense.api import addresses_api, blocks_api, entities_api

import matplotlib.pyplot as plt

from pprint import pprint

## Configure and create API endpoint

In [ ]:
configuration = graphsense.Configuration(
    host = "https://api.ikna.io",
    api_key = {'api_key': 'PASTE API KEY HERE'}
)

CURRENCY = 'btc'

## Case study #1

Let's have a look at some properties of block 500000 in the BTC blockchain.

In [3]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = blocks_api.BlocksApi(api_client)
    try:
        # Retrieve the list of all transactions in a block
        block_txs_obj = api_instance.list_block_txs(CURRENCY, 500000)
        pprint(block_txs_obj)
    except graphsense.ApiException as e:
        print("Exception when calling BlocksApi->list_block_txs: %s\n" % e)

Let's compute the paid fees and the ratio between fees and and total inputs for each transaction.

In [ ]:

block_tx_fees = [tx.total_input.value - tx.total_output.value
                 for tx in block_txs_obj if len(tx.inputs.value) > 0]
block_tx_inputs = [tx.total_input.value for tx in block_txs_obj if len(tx.inputs.value) > 0]
block_tx_fee_input_ratios = [f/i for f, i in zip(block_tx_fees, block_tx_inputs)]

Let's plot the results to visually identify some patterns.

In [ ]:
fs = 14
n_subplots = 3
fig, axs = plt.subplots(n_subplots, figsize=(15, 7))
axs[0].plot(block_tx_fees, 'b', label='Tx Fee Paid')

axs[1].plot(block_tx_inputs, 'g', label='Total Tx Input')

axs[2].plot(block_tx_fee_input_ratios, 'y', label='Fee/Input Ratio')

for i in range(n_subplots):
    axs[i].legend(fontsize=fs)
    axs[i].tick_params(labelsize=fs)
plt.show()

There is something weird happening around tx number 250 and around tx number 500 where the ratio is constant and close to 1. Let's check the numbers in the block. 

In [ ]:
delta = 3
print('index, ratio, input, fee')
for i, el in enumerate(block_tx_fee_input_ratios):
    if el > 0.9 and ((i > 250 - delta and i < 250 + delta) or (i > 500 - delta and i < 500 + delta)):
        print(i, el, block_tx_inputs[i], block_tx_fees[i])

Let's check how many transactions follow this pattern.

In [ ]:
from statistics import mode

In [ ]:
ratio_mode = mode(block_tx_fee_input_ratios)
ratio_mode

In [ ]:
n_txs_with_pattern = len([el for el in block_tx_fee_input_ratios if el == ratio_mode])
n_txs_with_pattern

In [ ]:
ratio_txs_with_pattern = 100*n_txs_with_pattern/len(block_txs_obj)
ratio_txs_with_pattern

We have 177 transactions in block 500000 (~6.55%) that follow a specific input-fee pattern and these have likely been performed by the same actor.

## Case Study #2

Let's play with an address and its entity and see what we can discover about their relations.

In [ ]:
address = '18NSAybFEhLa6AG3gxEN1hiaRJoSvvbnDb'

with graphsense.ApiClient(configuration) as api_client:
    api_instance = addresses_api.AddressesApi(api_client)
    try:
        # Retrieve the address object
        address_obj = api_instance.get_address(CURRENCY, address)
        # pprint(address_obj)
        entity_obj = api_instance.get_address_entity(CURRENCY, address)
        # pprint(entity_obj)
    except graphsense.ApiException as e:
        print("Exception when calling AddressesApi: %s\n" % e)

Let's check some simple properties of the address and of the entity and then compare their number of incoming and outgoing transactions.

In [ ]:
address_obj.total_received

In [ ]:
address_obj.balance

In [ ]:
entity_obj.no_addresses

In [ ]:
from datetime import datetime

In [ ]:
print('First Tx:', datetime.utcfromtimestamp(entity_obj.first_tx.timestamp).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
print('Last Tx:', datetime.utcfromtimestamp(entity_obj.last_tx.timestamp).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
address_obj.no_incoming_txs, entity_obj.no_incoming_txs

In [ ]:
address_obj.no_outgoing_txs, entity_obj.no_outgoing_txs

It looks like this address was used in all the incoming and outgoing transactions of the entity. Let's discover more about this entity.

In [ ]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = entities_api.EntitiesApi(api_client)
    try:
        # Retrieve the entity's tags
        entity_tags = api_instance.list_address_tags_by_entity(CURRENCY, entity_obj.entity)
        # pprint(entity_tags)
    except graphsense.ApiException as e:
        print("Exception when calling EntitiesApi: %s\n" % e)

In [ ]:
entity_tags

Unfortunately, we have no tags for this entity. Let's check its incoming neighbors!

In [ ]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = entities_api.EntitiesApi(api_client)
    try:
        # Retrieve entity neighbors
        entity_neighbors_obj = api_instance.list_entity_neighbors('btc', entity_obj.entity,
                                                                  direction='in', include_labels=True)
    except graphsense.ApiException as e:
        print("Exception when calling EntitiesApi: %s\n" % e)

First, it's better to check how many incoming neighbors we are dealing with

In [ ]:
[e.entity for e in entity_neighbors_obj.neighbors]

Not too many, we can work with them. Let's see if we are lucky with the labels of the incoming neighbors!

In [ ]:
known_neighbors = [el for el in entity_neighbors_obj.neighbors if len(el.labels) > 0]
for neighbor in known_neighbors:
    print(f"{neighbor.labels[0]} (Entity ID: {neighbor.entity.entity})")

Bingo! We found a known entity, which sent money to our entity: **Poloniex.com**, which is a cryptoasset exchange.